### Inicialização do ambiante

Para testar o script de geração do mapa de calor, vamos gerar pontos de calor randomicos utlizando as coordenadas da cidade de Blumenau.

Latitude e longitude vem de http://bboxfinder.com/#-27.007138,-49.159698,-26.674459,-48.965378

Acurária do 3º decimal é de ~100m
https://gis.stackexchange.com/questions/8650/measuring-accuracy-of-latitude-and-longitude

In [52]:
# Carrega bibliotecas utilizadas
import folium
import numpy as np
import pandas as pd
from folium import plugins
import geopy
import string
import dask.dataframe as dd
from geopy.extra.rate_limiter import RateLimiter

# caminho para o arquivos original dos dados. Esse arquivos eh atualizado ao executar o notebook
ARQUIVO_DADOS = "/caminho/para/dados.csv"
# caminho do arquivo que ira contar endereco e coordenadas. Gerando ao executar o notebook
ARQUIVO_COORDENADAS = "/caminho/para/dados_coordenadas.csv"
CAMINHO_MAPA_TODOS_CID = "/caminho/para/mapa_todos_cids.html"
CAMINHO_MAPA_CID1 = "/caminho/pata/mapa_cid1.html"
CAMINHO_MAPA_CID2 = "/caminho/para/mapa_cid2.html"


In [2]:


# limpeza dos dados
dados = pd.read_csv(ARQUIVO_DADOS, sep=";")
shape_anterior = dados.shape

# ajusta ceps. Remove os "-" e mantem apenas aquelas com 8 digitos
dados['cep'] = dados["cep"].astype(str).map(lambda x: x.strip().replace('-','')).map(lambda x: x if len(x) == 8 else "")

# ajusta logradouro. Remove espaços em brancos e pontuações na extremidades do texto
dados['logradouro'] = dados["logradouro"].astype(str).map(lambda x: x.strip(string.whitespace + string.punctuation))
dados = dados.loc[dados["logradouro"] != "nan"]

#ajusta nome do bairro
dados["bairro"] = dados["bairro"].astype(str).map(lambda x: x.strip(string.whitespace + string.punctuation))
dados.loc[dados["bairro"] == "nan", "bairro"] = ""

# ajusta numero. Permite apenas numeros
def ajusta_numero(numero):
    """Permite apenas numeros com digitos"""
    numero = numero.strip(string.whitespace + string.punctuation)
    return numero if numero.isnumeric() else ""
    
dados['numero'] = dados["numero"].astype(str).map(ajusta_numero)

print(f"{shape_anterior[0] - dados.shape[0]} registros removidos")


# monta endereco
def monta_endereco_completo(linha):
    """Monta linha do endereço complete conforme dados disponiveis."""
    endereco = linha["logradouro"]
    endereco += "," + linha["numero"] if len(linha["numero"]) > 0 else ""
    endereco += "," + linha["bairro"] if len(linha["bairro"]) > 0 else ""
    endereco += ", Blumenau, Santa Catarina, Brasil"
    linha["endereco_completo"] = endereco.lower()

    endereco_sem_numero = linha["logradouro"]
    endereco_sem_numero += "," + linha["bairro"] if len(linha["bairro"]) > 0 else ""
    endereco_sem_numero += ", Blumenau, Santa Catarina, Brasil"
    linha["endereco_sem_numero"] = endereco_sem_numero.lower()
    return linha


# colocar valores default para as coordenadas
dados["latitude"] = 0
dados["longitude"] = 0
dados["endereco_completo"] = ""
dados["endereco_sem_numero"] = ""

##procura latitude e longitude dos endereços
ddf = dd.from_pandas(dados, npartitions=12)
# monta endereco
res = ddf.map_partitions(lambda df: df.apply(monta_endereco_completo,axis=1)).compute(scheduler='processes')

# grava o arquivo com as coordenadas
res.to_csv(ARQUIVO_DADOS,sep=";", index=False)


0 registros removidos


In [8]:
dados = pd.read_csv(ARQUIVO_DADOS, sep=";")

# usa API da Azure
localizador = geopy.geocoders.AzureMaps("PUT YOUR KEY HERE")

def get_coordenadas(linha):
    """Função que busca as coordenadas utilizando o localizador definido."""
    if linha["latitude"] and linha["latitude"] != 0 and linha["longitude"] and linha["longitude"] != 0:
        # não precisamos buscar coordenadas que já temos
        return linha
    try:
       # monta um endereco completo para ser utlizado pelo buscador de coordenadas
        localizacao = localizador.geocode(linha["endereco_sem_numero"])
        if localizacao:
            linha["latitude"] = localizacao.latitude
            linha["longitude"] = localizacao.longitude
    except Exception as e:
        print(f"Error: {linha['endereco_sem_numero']}. {e}")
    finally:
        print(f"{linha['endereco_sem_numero']} -> OK")
        return linha

# procure apenas pelos enderecos sem o numero na rua. Assim, temos um numero menor de registros.
enderecos = dados[["endereco_sem_numero", "latitude", "longitude"]]
#remove enderecos duplicados
enderecos.drop_duplicates(["endereco_sem_numero"], inplace=True)
#enderecos = enderecos[:15]

# busca coordenadas utilizando o geolocalizador
geocode = RateLimiter(get_coordenadas, min_delay_seconds=0)
ddf = dd.from_pandas(enderecos, npartitions=10)
res = ddf.map_partitions(lambda df: df.apply(get_coordenadas,axis=1)).compute(scheduler='processes') 
res.to_csv(ARQUIVO_COORDENADAS, sep=";", index=False)


/home/jvanz/heatmap-covid19/.venv/lib64/python3.6/site-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,endereco_sem_numero,latitude,longitude
0,"imperatriz leopoldina,velha, blumenau, santa c...",0,0
2,"santa maria,progresso, blumenau, santa catarin...",0,0
3,"vale do selke,testo salto, blumenau, santa cat...",0,0
4,"maria nuss,testo salto, blumenau, santa catari...",0,0
5,"elfonso dickmann,itoupavazinha, blumenau, sant...",0,0
6,"engenheiro paul werner,itoupava seca, blumenau...",0,0
7,"dos cacadores,velha central, blumenau, santa c...",0,0
8,"leopoldo roters,agua verde, blumenau, santa ca...",0,0
9,"santa efigenia,itoupava norte, blumenau, santa...",0,0
10,"15 de novembro,centro, blumenau, santa catarin...",0,0


### Geração do mapa de calor

In [49]:
# Carrega coordenadas dos pontos de calor do arquivo
dados = pd.read_csv(ARQUIVO_DADOS, sep=";")
enderecos = pd.read_csv(ARQUIVO_COORDENADAS, sep=";")
dados = dados.merge(enderecos, left_on="endereco_sem_numero", right_on="endereco_sem_numero", validate="many_to_one")
dados = dados.drop(['latitude_x', 'longitude_x'], axis=1)
dados = dados.rename(columns={"latitude_y": "latitude", "longitude_y": "longitude"})

# remove coordenadas de locais muitos distantes
# limite de "Blumenau": -27.007138,-49.159698,-26.674459,-48.965378
dados = dados[((dados.latitude <= -26.674459) & (dados.latitude >= -27.007138)) & ((dados.longitude >= -49.159698) &  (dados.longitude <= -48.965378))]

# pega somente as coordenadas e ignora outras colunas
coordenadas = dados[["latitude", "longitude"]]

long_min =  coordenadas.longitude.min()
long_max =  coordenadas.longitude.max()
lat_min = coordenadas.latitude.min()
lat_max = coordenadas.latitude.max()

# Plota o gráfico do mapa de calor com todos os cid
m = folium.Map(
    [np.mean([lat_min, lat_max]), np.mean([long_min, long_max])],
    zoom_start=11
)

m.add_child(
    plugins.HeatMap(coordenadas, radius=15)
)

m.save(CAMINHO_MAPA_TODOS_CID)

In [50]:
# filtra apenas os registros com CID 1
cid1 = dados[(dados.age == 1)]
coordenadas = cid1[["latitude", "longitude"]]

long_min =  coordenadas.longitude.min()
long_max =  coordenadas.longitude.max()
lat_min = coordenadas.latitude.min()
lat_max = coordenadas.latitude.max()

# Plota o gráfico do mapa de calor com todos os cid
m = folium.Map(
    [np.mean([lat_min, lat_max]), np.mean([long_min, long_max])],
    zoom_start=11
)

m.add_child(
    plugins.HeatMap(coordenadas, radius=15)
)

m.save(CAMINHO_MAPA_CID1)

In [51]:
# filtra apenas os registros com CID 2
cid2 = dados[(dados.age == 2)]
coordenadas = cid2[["latitude", "longitude"]]

long_min =  coordenadas.longitude.min()
long_max =  coordenadas.longitude.max()
lat_min = coordenadas.latitude.min()
lat_max = coordenadas.latitude.max()

# Plota o gráfico do mapa de calor com todos os cid
m = folium.Map(
    [np.mean([lat_min, lat_max]), np.mean([long_min, long_max])],
    zoom_start=11
)

m.add_child(
    plugins.HeatMap(coordenadas, radius=15)
)

m.save(CAMINHO_MAPA_CID2)